In [ ]:
!pip install dabl

In [104]:
import os
import json
import ast
import dabl
import math
import pandas as pd

In [105]:
os.getcwd()


'e:\\Workspaces\\SASRec.pytorch\\notebooks'

In [106]:
ROOT_DIR = os.path.join(os.getcwd(), "..", "data", "steam")

In [107]:
ROOT_DIR

'e:\\Workspaces\\SASRec.pytorch\\notebooks\\..\\data\\steam'

In [108]:
os.path.exists(os.path.join(ROOT_DIR, "steam_games.json"))

True

In [109]:
with open(os.path.join(ROOT_DIR, "steam_games.json")) as json_file:
    lines = json_file.read().splitlines()
records = [ast.literal_eval(line) for line in lines]

In [110]:
raw_df = pd.DataFrame(records)

In [111]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   discount_price  225 non-null    float64
 8   reviews_url     32133 non-null  object 
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  bool   
 12  id              32133 non-null  object 
 13  developer       28836 non-null  object 
 14  sentiment       24953 non-null  object 
 15  metascore       2677 non-null   object 
dtypes: bool(1), float64(1), object(14)
memory usage: 3.7+ MB


In [180]:
df = raw_df[['id', 'sentiment', 'genres', 'specs', 'price', 'early_access']]

In [181]:
#fix sentiment
sentiments = ['Unknown', 'Overwhelmingly Negative', "Very Negative", "Negative", "Mostly Negative", "Mixed", "Mostly Positive", "Positive" , "Very Positive" ,"Overwhelmingly Positive"]
sentiments_dict = {
    'Unknown': -1,
    'Overwhelmingly Negative': 0,
    'Very Negative': 1,
    'Negative': 2,
    'Mostly Negative': 3,
    'Mixed': 4,
    'Mostly Positive': 5,
    'Positive': 6,
    'Very Positive': 7,
    'Overwhelmingly Positive': 8,
}
df['sentiment'] = df['sentiment'].fillna('Unknown')
df['sentiment'] = df['sentiment'].apply(lambda x: x if x in sentiments else 'Unknown')
df['sentiment'] = df['sentiment'].apply(lambda x: sentiments_dict[x])
df['sentiment'] = df['sentiment'].astype(int)

<ipython-input-181-261ac1949ad6>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['sentiment'].fillna('Unknown')
<ipython-input-181-261ac1949ad6>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['sentiment'].apply(lambda x: x if x in sentiments else 'Unknown')
<ipython-input-181-261ac1949ad6>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [182]:
#fix genres
genres = set()
for item in list(raw_df["genres"].dropna()):        
    genres = genres | set(item)
print(len(genres))

def is_in_genere(genre, x):
    try:
        return True if genre in x else False
    except:
        return False

for genre in genres:
    df[f'genre_{genre}'] = df['genres'].apply(lambda x: is_in_genere(genre=genre, x=x))

df.drop(columns=['genres'],inplace=True)

22
<ipython-input-182-7c76aea05057>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'genre_{genre}'] = df['genres'].apply(lambda x: is_in_genere(genre=genre, x=x))


In [183]:
#fix specs
specs = set()
for item in list(raw_df["specs"].dropna()):        
    specs = specs | set(item)
print(len(specs))

def is_in_spec(spec, x):
    try:
        return True if spec in x else False
    except:
        return False

for spec in specs:
    df[f'spec_{spec}'] = df['specs'].apply(lambda x: is_in_spec(spec, x))

df.drop(columns=['specs'],inplace=True)

40


In [184]:
#fix price
strange_prices = ['Free to Play', 'nan','Free','Free Demo', 'Play for Free!', 'Install Now', 'Play WARMACHINE: Tactics Demo', 'Free Mod', 'Install Theme', 'Third-party', 'Play Now', 'Free HITMAN™ Holiday Pack', 'Play the Demo', 'Starting at $499.00', 'Starting at $449.00', 'Free to Try', 'Free Movie', 'Free to Use']
free_prices = ['Free to Play', 'Free', 'Free Demo', 'Play for Free!', 'Install Now', 'Play WARMACHINE: Tactics Demo', 'Free Mod', 'Install Theme', 'Play Now', 'Free HITMAN™ Holiday Pack', 'Play the Demo', 'Free to Try', 'Free Movie', 'Free to Use']
df['price'] = df['price'].apply(lambda x: 0 if x in free_prices else x)
df['price'] = df['price'].apply(lambda x: 0 if x == 'Third-party' else x)
df['price'] = df['price'].apply(lambda x: 499.0 if x == 'Starting at $499.00' else x)
df['price'] = df['price'].apply(lambda x: 449.0 if x == 'Starting at $449.00' else x)

In [185]:
types = dabl.detect_types(df)
print(types) 

                                 continuous  dirty_float  low_card_int  \
id                                     True        False         False   
sentiment                             False        False          True   
price                                 False         True         False   
early_access                          False        False         False   
genre_Adventure                       False        False         False   
...                                     ...          ...           ...   
spec_Includes level editor            False        False         False   
spec_Single-player                    False        False         False   
spec_Partial Controller Support       False        False         False   
spec_Standing                         False        False         False   
spec_Oculus Rift                      False        False         False   

                                 categorical   date  free_string  useless  
id                                 

In [187]:
df.to_csv(os.path.join(ROOT_DIR, "steam_games.csv"), index=False)